# Chapter 2:

<a target="_blank" href="https://colab.research.google.com/github/wandb/edu/blob/rag-irl/rag-advanced/notebooks/Chapter02.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Comprehensive Evaluation Strategies**

In this chapter, we will evaluate the two main components of a RAG pipeline - retriever and response generator.

Evaluating the retriever can be considered component evaluation. Depending on your RAG pipeline, there can be a few components and for ensuring robustness of your system, it is recommended to come up with evaluation for each components. 


In [ ]:
%load_ext autoreload
%autoreload 2

import json
import pathlib

import nest_asyncio

import wandb

nest_asyncio.apply()
import asyncio

import weave
from dotenv import load_dotenv

from scripts.utils import display_source

load_dotenv()

In this chapter we will also use W&B Weave for our evaluation purposes. The `weave.Evaluation` class is a light weight class that can be used to evaluate the performance of a `weave.Model` on a `weave.Dataset`. We will go into more details.

We will initialize a weave client which will track traces and evaluation scores.

In [ ]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

weave_client = weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

## Collecting data for evaluation

We used our [FAQs](https://docs.wandb.ai/guides/technical-faq) section from the docs website to build our evaluation set. 

The evaluation samples are logged as [`weave.Dataset`](https://wandb.github.io/weave/guides/core-types/datasets/). `weave.Dataset` enable you to collect examples for evaluation and automatically track versions for accurate comparisons. 

Below we will download the latest version locally with a simple API.

In [ ]:
# Easy eval dataset with 20 samples.
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

print("Number of evaluation samples: ", len(eval_dataset.rows))

Iterating through each sample is easy.

We have the question, ground truth answer and ground truth contexts.

In [ ]:
dict(eval_dataset.rows[0])

## Evaluating the Retriever

The fundamental idea of evaluating a retriever is to check how well the retrieved content matches the expected contents. For evaluating a RAG pipeline end to end, we need query and ground truth answer pairs. The ground truth answer must be grounded on some "ground" truth chunks. This is a search problem, it's easiest to start with tradiaional Information retrieval metrics.

You might already have access to such evaluation dataset depending on the nature of your application or you can synthetically build one. To build one you can retrieve random documents/chunks and ask an LLM to generate query-answer pairs - the underlying documents/chunks will act as your ground truth chunk.

In the sections below, we will look at different metrics that can be used to evaluate the retriever we built in the last chapter.

First let us download the chunked data (from chapter 1) and index it using a simple TFIDF based retriever.


In [ ]:
# Reload the data from Chapter 1
chunked_data = weave.ref('chunked_data:v0').get()
chunked_data.rows[:2]

We will import the `TFIDFRetriever` which we created in the last chapter and index the chunked data.

In [ ]:
from scripts.retriever import TFIDFRetriever

display_source(TFIDFRetriever)

retriever = TFIDFRetriever()
retriever.index_data(chunked_data.rows)

## Metrics to evaluate retriever

We can evaluate a retriever using traditional ML metrics. We can also evaluate by using a powerful LLM (next section).

Below we are importing both traditional metrics and LLM as a judge metric from the `scripts/retrieval_metrics.py` file.

In [ ]:
from scripts.retrieval_metrics import IR_METRICS, LLM_METRICS as RETRIEVAL_LLM_METRICS
from scripts.utils import display_source

Let us first understand the basic traditional metrics we will be using. Each metric expects a `model_output` which is a list of retrieved chunks from the retriever and `contexts` which is a list of ground truth contexts.

- `compute_hit_rate`: The Hit Rate metric measures the proportion of retrieved documents that are relevant. We compute the proportion of relevant documents retrieved out of the total documents retrieved.
- `compute_mrr`: MRR or mean reciprocal ranking measure how quickly a retrieval system can return the first relevant document in response to a query. Reciprocal ranking is computed by taking `1/rank` where rank is the position of the retrieved chunk in the ground truth chunks. MRR is the mean of these reciprocal ranks.
- 

In [ ]:
for scorer in IR_METRICS:
    display_source(scorer)

#### Evaluating retrieval on other metrics

In [ ]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset.rows,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k":5}
)
retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))

### Using an LLM as a Retrieval Judge

**ref: https://arxiv.org/pdf/2406.06519**

How do we evaluate if we don't have any ground truth? 

We can use a powerful LLM as a judge to evaluate the retriever. 


In [ ]:
for metric in RETRIEVAL_LLM_METRICS:
    display_source(metric)

In [ ]:
retrieval_evaluation = weave.Evaluation(
    name="LLM_Judge_Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=RETRIEVAL_LLM_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k":5}
)
retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))

## Evaluating the Response

In [ ]:
from scripts.rag_pipeline import SimpleRAGPipeline
from scripts.response_generator import SimpleResponseGenerator

INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
rag_pipeline = SimpleRAGPipeline(retriever=retriever, response_generator=response_generator, top_k=5)

In [ ]:
from scripts.response_metrics import NLP_METRICS, LLM_METRICS as RESPONSE_LLM_METRICS
for scorer in NLP_METRICS:
    display_source(scorer)

In [ ]:
response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=NLP_METRICS[:-1],
    preprocess_model_input=lambda x: {"query": x["question"]})
response_scores = asyncio.run(response_evaluations.evaluate(rag_pipeline))

### Using an LLM as a Response Judge

Some metrics cannot be defined objectively and are particularly useful for more subjective or complex criteria.
We care about correctness, faithfulness, and relevance.

- **Answer Correctness** - Is the generated answer correct compared to the reference and thoroughly answers the user's query?
- **Answer Relevancy** - Is the generated answer relevant and comprehensive?
- **Answer Factfulness** - Is the generated answer factually consistent with the context document?


In [ ]:
for metric in RESPONSE_LLM_METRICS:
    display_source(metric)

In [ ]:
correctness_evaluations = weave.Evaluation(
    name="Correctness_Evaluation",
    dataset=eval_dataset,
    scorers=RESPONSE_LLM_METRICS, 
    preprocess_model_input=lambda x: {"query": x["question"]})
response_scores = asyncio.run(correctness_evaluations.evaluate(rag_pipeline))

## Exercise

1. Implement the `Relevance` and `Faithfulness` evaluators and evaluate the pipeline on all the dimensions.
2. Generate and share a W&B report with the following sections in the form of tables and charts:
    
    - Summary of the evaluation
    - Retreival Evaluations
        - IR Metrics
        - LLM As a Retrieval Judge Metric
    - Response Evalations
        - Traditional NLP Metrics
        - LLM Judgement Metrics
    - Overall Evalations
    - Conclusion
